In [6]:
import pandas as pd
df = pd.read_pickle('../data/val.pkl')

# Отбираю для теста retrival-ы c открытокого leaderboard https://github.com/avidale/encodechka

пробуем прежде всего лидера, мультиязычная модель

я буду скачивать через git lfs, потому что у меня мало памяти. Можно напрямую вставлять c hf. Я заглядываю в лидерборт и копирую название в huggingface

In [24]:
!git lfs install
!git clone https://huggingface.co/intfloat/multilingual-e5-large

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'multilingual-e5-large'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 41 (delta 6), reused 3 (delta 3), pack-reused 31
Unpacking objects: 100% (41/41),

In [4]:
import torch.nn.functional as F
import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.

tokenizer = AutoTokenizer.from_pretrained('multilingual-e5-large')
model = AutoModel.from_pretrained('multilingual-e5-large')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [5]:
from collections import Counter, defaultdict
import numpy as np
import gc
import torch

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    
cleanup()

In [6]:
# Инициализируем список для хранения всех векторов предложений
from tqdm.auto import tqdm
import torch
all_sentence_embeddings1 = []
batch_size = 50
# Получаем количество сэмплов
num_samples = len(df['position'].tolist())
  
# Проходим по всем сэмплам по batch_size
for i in tqdm(range(0, num_samples, batch_size)):
    # Выбираем текущий батч
    sentence_batch = df['position'].tolist()[i:i+batch_size]

    # Tokenize sentences 
    batch_dict = tokenizer(sentence_batch, max_length=215, padding=True, truncation=True, return_tensors='pt').to(device)
    
    outputs = model(**batch_dict)
    sentence_embeddings1 = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    sentence_embeddings1 = F.normalize(sentence_embeddings1, p=2, dim=1)

    # Добавляем вектора предложений текущего батча в список
    all_sentence_embeddings1.append(sentence_embeddings1.cpu().detach().numpy())
    del sentence_embeddings1
    cleanup()
    

# Конкатенируем все вектора предложений
sentence_embeddings1 = np.concatenate(all_sentence_embeddings1, axis=0)

  0%|          | 0/34 [00:00<?, ?it/s]

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
#pas@5
cum_sum_1 = 0
RR = 0
for index in tqdm(df.index):
    question = df['description'][index]
    
    # это лайфхак для повторений топиков, я хочу, чтобы если модель найдет не свою строку, но такойже топик
    # мы считали ее ответ за правильный в метрике
    mask = df['position'] == df['position'][index]

    new_df = df.copy()
    new_df.index = df.index.where(~mask, index)

    encoded_question = tokenizer(question, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model(**encoded_question)
    embeddings1 = average_pool(outputs.last_hidden_state, encoded_question['attention_mask'])
    embeddings1 = F.normalize(embeddings1, p=2, dim=1)
    cos_similarities = cosine_similarity(embeddings1.cpu().detach().numpy(), sentence_embeddings1)[0]
    new_df["rank"]= cos_similarities
    rank_s = new_df["rank"].sort_values(ascending=False)
    if index in rank_s[:1].index:
        cum_sum_1 += 1
    RR += 1/(list(rank_s.index).index(index)+1)

  0%|          | 0/1653 [00:00<?, ?it/s]

In [9]:
print(f"Результат работы p@1: {int(100*cum_sum_1/len(df))} %")
print(f"Mean reciprocal rank: {round((1/len(df))*RR,2)}")

Результат работы p@1: 2 %
Mean reciprocal rank: 0.06


# Это было первое место в лидерборде, теперь попробуем некоторые модели ниже, у них у всех общий инференс, поэтому я буду скачивать, и когда буду пробовать новую, я просто буду менять имя вновь загруженной модели. То есть из таблицы этто люое имя, кроме уже ранее задействованного

In [7]:
import pandas as pd
df = pd.read_pickle('../data/val.pkl')

In [12]:
!git lfs install
!git clone https://huggingface.co/sentence-transformers/LaBSE

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'LaBSE'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 54 (delta 16), reused 54 (delta 16), pack-reused 0
Unpacking objects: 100% (54/54), 7.12 MiB | 

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('paraphrase_documents_v3')
model = AutoModel.from_pretrained('paraphrase_documents_v3')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to('cuda')

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [22]:
# Инициализируем список для хранения всех векторов предложений
from tqdm.auto import tqdm
import torch
all_sentence_embeddings1 = []
batch_size = 100
# Получаем количество сэмплов
num_samples = len(df['position'].tolist())
  
# Проходим по всем сэмплам по batch_size
for i in tqdm(range(0, num_samples, batch_size)):
    # Выбираем текущий батч
    sentence_batch = df['position'].tolist()[i:i+batch_size]

    # Tokenize sentences 
    encoded_input = tokenizer(sentence_batch, padding=True, truncation=True, return_tensors='pt').to(device)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Perform pooling. In this case, max pooling.
    sentence_embeddings1 = mean_pooling(model_output, encoded_input['attention_mask'])

    # Добавляем вектора предложений текущего батча в список
    all_sentence_embeddings1.append(sentence_embeddings1)

# Конкатенируем все вектора предложений
sentence_embeddings1 = torch.cat(all_sentence_embeddings1, dim=0)

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
#pas@5
cum_sum_1 = 0
RR = 0
for index in tqdm(df.index):
    question = df['description'][index]
     # это лайфхак для повторений топиков, я хочу, чтобы если модель найдет не свою строку, но такойже топик
    # мы считали ее ответ за правильный в метрике
    mask = df['position'] == df['position'][index]

    new_df = df.copy()
    new_df.index = df.index.where(~mask, index)

    encoded_question = tokenizer(question, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        model_output = model(**encoded_question)
    sentence_embeddings = mean_pooling(model_output, encoded_question['attention_mask'])
    cos_similarities = cosine_similarity(sentence_embeddings.detach().cpu().numpy(), sentence_embeddings1)[0]
    new_df["rank"]= cos_similarities
    rank_s = new_df["rank"].sort_values(ascending=False)
    if index in rank_s[:1].index:
        cum_sum_1 += 1
    RR += 1/(list(rank_s.index).index(index)+1)

print(f"Результат работы p@1: {int(100*cum_sum_1/len(df))} %")
print(f"Mean reciprocal rank: {round((1/len(df))*RR,2)}")

  0%|          | 0/1653 [00:00<?, ?it/s]

Результат работы p@1: 2 %
Mean reciprocal rank: 0.05


In [ ]:
cos_similarities = util.cos_sim(sentence_embeddings, sentence_embeddings1)
df["rank"]= cos_similarities[0].detach().cpu()